# Pandas and duckdb

## Task 0 - Load the sakila sqlite database into a duckdb database file

In [ ]:
import duckdb 
from pathlib import Path

with duckdb.connect("data/sakila.duckdb") as conn:
    description = conn.sql("DESC").df()

description

In [ ]:
# import duckdb
# from pathlib import Path

# duckdb_path = "data/sakila.duckdb"
# Path(duckdb_path).unlink(missing_ok=True)

# with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
#     conn.sql(ingest_script.read())

#     description = conn.sql("DESC;").df()
#     films = conn.sql("FROM film;").df()

# films.head()

# Read all data into dictionary of pandas dataframes

In [ ]:
dfs = {}

with duckdb.connect(duckdb_path) as conn:
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM {name};").df()
        duckdb.register(name,dfs[name])

dfs.keys()

## Task 1

In [ ]:
long_movies = duckdb.sql("""
SELECT title, length AS duration
FROM films 
WHERE length > 180; 
""").df()

long_movies.head()

In [ ]:
love_word = duckdb.sql("""
SELECT title, rating, length, description 
FROM films
WHERE title ILIKE '%love%'; 
""").df()

love_word.head()

In [ ]:
aggregate_function = duckdb.sql("""
SELECT 
    MIN(length) AS min_length_movies, 
    ROUND(AVG(length)) AS mean_length_movies, 
    MEDIAN(length) AS median_length_movies, 
    MAX(length) AS max_length_movies
FROM films
""").df()


aggregate_function

In [ ]:
rental_rate = duckdb.sql("""
SELECT title, rental_rate, rental_duration, (rental_rate / rental_duration) AS cost_per_day                
FROM films
ORDER BY cost_per_day DESC;
""").df()

rental_rate.head(10)

In [ ]:
description

In [ ]:
top_10_actors = duckdb.sql("""
SELECT a.actor_id, a.first_name, a.last_name, COUNT(*) AS film_count                
FROM films_joined 
JOIN film_actor fa ON a.actor_id = fa.actor_id
GROUP BY a.actor_id, a.first_name, a.last_name
ORDER BY fil_count DESC;
""").df()

top_10_actors